In [16]:
import torch
import numpy as np
import torch.nn as nn


In [5]:
x = torch.Tensor(2,2)
print(x)
x = torch.Tensor([[1, 2], [3, 4]])
print(x)

tensor([[0., 0.],
        [0., 0.]])
tensor([[1., 2.],
        [3., 4.]])


In [6]:
x = [[1, 2], [3, 4]]
print(x)
x = np.array(x)
print(x)
x = torch.from_numpy(x)
print(x)

[[1, 2], [3, 4]]
[[1 2]
 [3 4]]
tensor([[1, 2],
        [3, 4]], dtype=torch.int32)


Autograd

In [13]:
x = torch.FloatTensor([[1, 2], [3, 4]])
print(x)
y = torch.FloatTensor([[1, 3], [5, 7]])
print(y)
y.requires_grad_(True)
print(y)
z = (x + y) + torch.FloatTensor([[1, 4], [3, 3]])
print(z)
print(x)
print(y)

tensor([[1., 2.],
        [3., 4.]])
tensor([[1., 3.],
        [5., 7.]])
tensor([[1., 3.],
        [5., 7.]], requires_grad=True)
tensor([[ 3.,  9.],
        [11., 14.]], grad_fn=<AddBackward0>)
tensor([[1., 2.],
        [3., 4.]])
tensor([[1., 3.],
        [5., 7.]], requires_grad=True)


In [12]:
x = torch.FloatTensor([[1, 2], [3, 4]])
print(x)
y = torch.FloatTensor([[1, 3], [5, 7]])
print(y)
y.requires_grad_(True)
print(y)

with torch.no_grad():
    z = (x + y) + torch.FloatTensor([[1, 4], [3, 3]])
    print(z)
    print(x)
    print(y)

tensor([[1., 2.],
        [3., 4.]])
tensor([[1., 3.],
        [5., 7.]])
tensor([[1., 3.],
        [5., 7.]], requires_grad=True)
tensor([[ 3.,  9.],
        [11., 14.]])


Feed forward

In [15]:
x = torch.FloatTensor([[1, 2, 3], [2, 3, 4]])
y = torch.FloatTensor([[1, 2], [2, 3], [3, 4]])
z = torch.FloatTensor([[1, 2], [3, 4]])

def linear(x, y, z):
    result = torch.mm(x, y) + z
    return result

print(linear(x, y, z))
# mm -> matrix multiplication
# bmm -> batch matrix multiplication
# matmul -> matrix, batch matrix multiplication

tensor([[15., 22.],
        [23., 33.]])


nn.Module

In [21]:
class mylinear(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()

        self.w = torch.FloatTensor(input_size, output_size)
        self.b = torch.FloatTensor(output_size)

    def forward(self, x):
        return torch.mm(x, self.w) + self.b
        
linear = mylinear(2, 4)
result = linear(torch.FloatTensor(4, 2))
print(result)

tensor([[0.0000e+00, 0.0000e+00, 6.6021e-07, 1.7185e-04],
        [1.9635e-10, 1.9979e-10, 6.6041e-07, 1.7185e-04],
        [0.0000e+00, 0.0000e+00, 6.6021e-07, 1.7185e-04],
        [0.0000e+00, 0.0000e+00, 6.6021e-07, 1.7185e-04]])


In [23]:
params = [p for p in linear.parameters()]
print(params)

[]


빈 배열이 나오는 이유는 학습 가능한 파라미터가 없기 때문
-> parameter객체로 텐서를 감싸야한다.

In [45]:
class mylinear(nn.Module):
    def __init__(self, input_size, output_size):
        super(mylinear, self).__init__()

        self.w = nn.Parameter(torch.FloatTensor(input_size, output_size))
        self.b = nn.Parameter(torch.FloatTensor(output_size))
        print('self.b', self.b)
    def forward(self, x):
        return torch.mm(x, self.w) + self.b

linear = mylinear(2, 4)
result = linear(torch.FloatTensor(4, 2))
print(result)

self.b Parameter containing:
tensor([0., 0., 0., 0.], requires_grad=True)
tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [3.3221e+13, 4.3370e+28,        inf,        inf],
        [6.4999e-21, 1.4923e-09, 1.2102e+19, 3.3221e+13],
        [1.6546e-18, 1.2136e-07, 9.8422e+20, 2.7017e+15]],
       grad_fn=<AddBackward0>)


In [61]:
class mylinear(nn.Module):
    def __init__(self, input_size, output_size):
        super(mylinear, self).__init__()

        self.w = nn.Parameter(torch.FloatTensor([[1, 2], [3, 4]]))
        self.b = nn.Parameter(torch.FloatTensor([[5, 5], [5, 5]]))

    def forward(self, x):
        return torch.mm(x, self.w) + self.b

linear = mylinear(2, 4)
result = linear(torch.FloatTensor([[5, 6], [7, 8]]))
print(result)

tensor([[28., 39.],
        [36., 51.]], grad_fn=<AddBackward0>)


In [62]:
params = [p for p in linear.parameters()]
print(params)

[Parameter containing:
tensor([[1., 2.],
        [3., 4.]], requires_grad=True), Parameter containing:
tensor([[5., 5.],
        [5., 5.]], requires_grad=True)]


backward 역전파

In [63]:
objective = 50

loss = (objective - result.sum())**2
print('loss', loss)
print(loss.backward())

loss tensor(10816., grad_fn=<PowBackward0>)
None


In [66]:
linear = nn.Linear(2, 4)
result = linear(torch.FloatTensor(4, 2))
print(result)

tensor([[ 6.3614e-01,  2.5292e-01, -4.3508e-01, -4.7283e-01],
        [ 6.3614e-01,  2.5292e-01, -4.3508e-01, -4.7283e-01],
        [ 6.3614e-01,  2.5292e-01, -4.3508e-01, -4.7283e-01],
        [-1.4185e+20, -7.0060e+18,  1.2350e+20, -7.7721e+19]],
       grad_fn=<AddmmBackward>)


In [67]:
class Mymodel(nn.Module):
    def __init__(self, input_size, output_size):
        super(Mymodel, self).__init__()

        self.linear = nn.Linear(input_size, output_size, True)

    def forward(self, x):
        return self.linear(x)

def ground_truth(x):
    return 3 * x[: 0] + x[: 1] - 2 * x [: 2]

def train(model, x, y, optim):
    optim.zero_grad()

    y_hat = model(x)

    loss = ((y - y_hat)**2).sum() / x.size(0)
    
    loss.backward()

    optim.step()

    return loss.data

def valid(model, x, y, optim):
    with torch.no_grad():
        
        y_hat = model(x)

        loss = ((y - y_hat)**2).sum() / x.size(0)
        
        loss.backward()

        optim.step()

        return loss.data

In [68]:
batch_size = 1
n_epochs = 1000
n_iter = 10000

model = Mymodel(3, 1)
optim = torch.optim.SGD(model.parameters(), lr = 0.0001, momentum = 0.1)

print(model)

Mymodel(
  (linear): Linear(in_features=3, out_features=1, bias=True)
)


In [69]:
for epoch in range(n_epochs):
    avg_loss = 0

    for i in range(n_iter):
        x = torch.rand(batch_size, 3)
        y = ground_truth(x.data)

        loss = train(model, x, y, optim)

        avg_loss += loss
    avg_loss = avg_loss/n_iter

    x_valid = torch.FloatTensor([[.3, .2, .1]])
    y_valid = ground_truth(x_valid.data)

    valid_loss = 


x tensor([[0.0146, 0.6631, 0.1364]])
x.data tensor([[0.0146, 0.6631, 0.1364]])


KeyboardInterrupt: Interrupted by user